In [3]:
import pandas as pd
import requests

In [14]:
# read keys
import yaml
import os

def read_config():
    # Get the directory of the current script
    # script_dir = os.path.dirname(os.path.realpath(__file__))
    script_dir = "../src/"

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        try:
            configs = yaml.safe_load(stream)
            api_key = configs['amadeues_flights']['api_key']
            api_secret = configs['amadeues_flights']['api_secret']
            return api_key, api_secret
        except yaml.YAMLError as exc:
            print(exc)
            
    return api_key, api_secret


class SingletonToken:
    __token = None

    @classmethod
    def set_token(cls, token):
        cls.__token = token

    @classmethod
    def get_token(cls):
        return cls.__token

api_key, api_secret = read_config()

t09OfctGEcA7qDCHcp2m1gTZ9yN7Cgzu XpF14SwN2uWcfkA2


In [22]:
from amadeus import Client, ResponseError

amadeus = Client(client_id=api_key, client_secret=api_secret)

# Defining the parameters for the flight
params = {
    'originLocationCode': 'SYD',
    'destinationLocationCode': 'BKK',
    'departureDate': '2023-10-10',
    'adults': 1
    # add or modify other parameters according to your needs
}

try:
    response_flights = amadeus.shopping.flight_offers_search.get(**params)
    
except ResponseError as error:
    print(f"ResponseError occurred: {error}")
    print(f"Error code: {error.code}")
    print(f"Error message: {error.description}")


try:
    response_airline_lookup = amadeus.reference_data.airlines.get()

    
except ResponseError as error:
    print(f"ResponseError occurred: {error}")
    print(f"Error code: {error.code}")
    print(f"Error message: {error.description}")




In [23]:
df_flights = pd.json_normalize(response_flights.data)
df_airline_codes = pd.json_normalize(response_airline_lookup.data)

In [33]:
import langchain
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain.agents.agent_toolkits'

In [25]:
df_flights

,type,id,source,instantTicketingRequired,nonHomogeneous,oneWay,lastTicketingDate,lastTicketingDateTime,numberOfBookableSeats,itineraries,validatingAirlineCodes,travelerPricings,price.currency,price.total,price.base,price.fees,price.grandTotal,pricingOptions.fareType,pricingOptions.includedCheckedBagsOnly,price.additionalServices
0,flight-offer,1,GDS,False,False,False,2023-07-17,2023-07-17,9,"[{'duration': 'PT24H15M', 'segments': [{'depar...",[SQ],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,250.85,150.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",250.85,[PUBLISHED],True,NaN
1,flight-offer,2,GDS,False,False,False,2023-07-17,2023-07-17,9,"[{'duration': 'PT28H40M', 'segments': [{'depar...",[SQ],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,292.85,192.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",292.85,[PUBLISHED],True,NaN
2,flight-offer,3,GDS,False,False,False,2023-07-17,2023-07-17,9,"[{'duration': 'PT16H5M', 'segments': [{'depart...",[SQ],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,294.85,194.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",294.85,[PUBLISHED],True,NaN
3,flight-offer,4,GDS,False,False,False,2023-07-17,2023-07-17,9,"[{'duration': 'PT20H30M', 'segments': [{'depar...",[SQ],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,294.85,194.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",294.85,[PUBLISHED],True,NaN
4,flight-offer,5,GDS,False,False,False,2023-07-23,2023-07-23,9,"[{'duration': 'PT13H15M', 'segments': [{'depar...",[GP],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,315.01,178.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",315.01,[PUBLISHED],True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,flight-offer,74,GDS,False,False,False,2023-09-27,2023-09-27,9,"[{'duration': 'PT20H35M', 'segments': [{'depar...",[NH],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,5914.48,5545.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",5914.48,[PUBLISHED],True,NaN
74,flight-offer,75,GDS,False,False,False,2023-09-27,2023-09-27,9,"[{'duration': 'PT34H50M', 'segments': [{'depar...",[NH],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,5914.48,5545.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",5914.48,[PUBLISHED],True,NaN
75,flight-offer,76,GDS,False,False,False,2023-09-27,2023-09-27,9,"[{'duration': 'PT29H35M', 'segments': [{'depar...",[NH],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,5922.11,5545.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",5922.11,[PUBLISHED],True,NaN
76,flight-offer,77,GDS,False,False,False,2023-09-27,2023-09-27,9,"[{'duration': 'PT30H20M', 'segments': [{'depar...",[NH],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",EUR,5922.11,5545.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",5922.11,[PUBLISHED],True,NaN
